In [2]:
%matplotlib qt

In [3]:
import os
import sys
sys.path.append('/dls/science/groups/e02/Mohsen/code/Git_Repos/Merlin-Medipix/')
import epsic_tools.api as epsic
import numpy as np

In [4]:
def get_distance(p1, p2):
    x1 = p1[0]
    y1 = p1[1]
    x2 = p2[0]
    y2 = p2[1]
    
    dx = x2 - x1
    dy = y2 - y1
    return np.sqrt(dx ** 2  +dy **2)

In [5]:
import matplotlib.pyplot as plt
def plot_kdtree_results(paired_list, FN, FP, unlabeled_experiment=None, truth=None):
    

    if type(FN) is list:
        FN = np.asarray(FN)
    if type(FP) is list:
        FP = np.asarray(FP)
        
    if (unlabeled_experiment is None) and (truth is None):
        fig, axs = plt.subplots(nrows=1, ncols=1)
        detected_list = []
        for i in range(len(paired_list)):
            detected_list.append(paired_list[i][0])
        detected_arr = np.asarray(detected_list)
        axs.scatter(detected_arr[:,0], detected_arr[:,1], c='g', marker='s', label='matched')
        axs.scatter(FN[:,0], FN[:,1], marker='o', c = 'r', label = 'false_neg')
        axs.scatter(FP[:,0], FP[:,1], marker='o', c = 'b', label = 'false_pos')
        axs.set_xticks([])
        axs.set_yticks([])
        axs.legend()
        
        
    else:
        if type(unlabeled_experiment) is list:
            unlabeled_experiment = np.asarray(unlabeled_experiment)
        if type(truth) is list:
            truth = np.asarray(truth)
        fig, axs = plt.subplots(nrows=1, ncols=2)
        detected_list = []
        for i in range(len(paired_list)):
            detected_list.append(paired_list[i][1][0])
        detected_arr = np.asarray(detected_list)
        axs[0].scatter(detected_arr[:,0], detected_arr[:,1], c='g', marker='s', label='matched')
        axs[0].scatter(FN[:,0], FN[:,1], marker='x', c = 'r', label = 'false_neg')
        axs[0].scatter(FP[:,0], FP[:,1], marker='o', c = 'm', label = 'false_pos')
    #     axs[0].scatter(truth_arr[:,0], truth_arr[:,1],s =3 ,marker = 'o', c = 'r', label = 'truth')

        axs[1].scatter(truth[:,0], truth[:,1], s =3 ,marker = 'o', c = 'r',label = 'truth')
        axs[1].scatter(unlabeled_experiment[:,0], unlabeled_experiment[:,1], marker = 'x', label = 'experiment_unlabeled')
        axs[0].legend()
        axs[1].legend()
    return 


# Iterating over the fitting results 

In [9]:
# C-C bond is approx. 8 pixels and recon pixel size is 1.7e-11 m

In [6]:
# fitting_folder = '/dls/science/groups/e02/Mohsen/code/Git_Repos/Staff-notebooks/ptyREX_sim_matrix/fitting_results/graphene_512_64matrix/'
fitting_folder = '/dls/science/groups/e02/Mohsen/code/Git_Repos/Staff-notebooks/ptyREX_sim_matrix/fitting_results/graphene_512_64matrix_4pix_searchrad/'
truth = np.load('/dls/science/groups/e02/Mohsen/code/Git_Repos/Staff-notebooks/ptyREX_sim_matrix/ground_truth_positions.npy')
truth_pos = truth[:,:2]
# test = '/dls/science/groups/e02/Mohsen/code/Git_Repos/Staff-notebooks/ptyREX_sim_matrix/fitting_results/test'
search_rad = 4.0
for file in os.listdir(fitting_folder):
    file_path = os.path.join(fitting_folder, file)
    data_dict = epsic.ptycho_utils.load_dict_from_hdf5(file_path)
    exp_pos = data_dict['atom_pos'][:,:2]
    nn_dict = epsic.ptycho_utils.kdtree_NN(exp_pos, truth_pos, search_rad)
    data_dict.update({'NN_Results': nn_dict})
    epsic.ptycho_utils.save_dict_to_hdf5(data_dict, os.path.join(fitting_folder,file))

# Plotting results

In [5]:
hdf_folder = '/dls/science/groups/e02/Mohsen/code/Git_Repos/Staff-notebooks/ptyREX_sim_matrix/fitting_results/graphene_512_64matrix_4pix_searchrad/' 

In [6]:
hdf_files = []
for file in os.listdir(hdf_folder):
    hdf_files.append(os.path.join(hdf_folder, file))

In [7]:
# sorting the json files
conv_angles = [0.016, 0.020, 0.024, 0.030, 0.040, 0.050, 0.064, 0.084]
# diff_overlap = [-22.5, 1.99, 18.33, 34.70, 51.00, 60.80, 69.37, 76.67]
# real_probe_overlap = [0, 5, 15, 35, 60, 70, 80, 90]


data_list_of_dicts = []
for file in hdf_files:
    j_dict = epsic.ptycho_utils.load_dict_from_hdf5(file)

    for i, angle in enumerate(conv_angles):
        data_list_of_dicts.append([])
        if j_dict['process']['common']['probe']['convergence'] == angle:
            data_list_of_dicts[i].append(j_dict)
                
for i, angle in enumerate(conv_angles):
    print(angle)
    data_list_of_dicts[i].sort(key=lambda e: e['process']['common']['scan']['dR'][0], reverse = True)


0.016
0.02
0.024
0.03
0.04
0.05
0.064
0.084


In [8]:
data_list_of_dicts[6][6]['NN_Results']['RMSE']

0.1754256989157329

In [9]:
plot_kdtree_results(data_list_of_dicts[6][6]['NN_Results']['TP_list'],
                   data_list_of_dicts[6][6]['NN_Results']['FN_list'],
                   data_list_of_dicts[6][6]['NN_Results']['FP_list'])

In [8]:
import matplotlib.pyplot as plt
def plot_kdtree_results_matrix(paired_list, FN, FP, axs=None, **plt_kwargs):
    

    if type(FN) is list:
        FN = np.asarray(FN)
    if type(FP) is list:
        FP = np.asarray(FP)
        
    if axs is None:
        axs = plt.gca()

    detected_list = []
    for i in range(len(paired_list)):
        detected_list.append(paired_list[i][0])
    detected_arr = np.asarray(detected_list)
    axs.scatter(detected_arr[:,0], detected_arr[:,1], c='g', marker='s', label='matched', **plt_kwargs)
    axs.scatter(FN[:,0], FN[:,1], marker='o', c = 'r', label = 'false_neg', **plt_kwargs)
    axs.scatter(FP[:,0], FP[:,1], marker='o', c = 'b', label = 'false_pos', **plt_kwargs)
    axs.set_xticks([])
    axs.set_yticks([])
#         axs.legend()
    return 


In [9]:
# NN results
rows = len(data_list_of_dicts[0])
cols = len(conv_angles)
#print(rows, cols)
# fig, axs = plt.subplots(nrows=rows, ncols=cols, sharex=True, sharey=True,figsize=(8, 11))
fig, axs = plt.subplots(nrows=rows, ncols=cols, figsize=(8, 11))

rmse_list = []
precision_list = []
recall_list = []

conv_angles = []
step_sizes = []

for idx in range(rows):    
      for idy in range(cols): 
            try:
                plot_kdtree_results_matrix(data_list_of_dicts[idy][idx]['NN_Results']['TP_list'],
                   data_list_of_dicts[idy][idx]['NN_Results']['FN_list'],
                   data_list_of_dicts[idy][idx]['NN_Results']['FP_list'],
                                          axs=axs[idx,idy], s =1)
                axs[idx,idy].set_title(str(np.round(data_list_of_dicts[idy][idx]['NN_Results']['RMSE'],2)) + '  ' + \
                                       str(np.round(data_list_of_dicts[idy][idx]['NN_Results']['Precision'],2)) + '  ' + \
                                       str(np.round(data_list_of_dicts[idy][idx]['NN_Results']['Recall'],2)), fontsize = 8)
                
                rmse_list.append(data_list_of_dicts[idy][idx]['NN_Results']['RMSE'])
                precision_list.append(data_list_of_dicts[idy][idx]['NN_Results']['Precision'])
                recall_list.append(data_list_of_dicts[idy][idx]['NN_Results']['Recall'])
                
                step_sizes.append(data_list_of_dicts[idy][idx]['process']['common']['scan']['dR'][0])
                conv_angles.append(data_list_of_dicts[idy][idx]['process']['common']['probe']['convergence'])

            except IndexError:
                pass
fig.tight_layout(pad = 1.0)  

In [83]:
plt.close('all')

In [10]:
min(rmse_list)

0.05760031729525667

In [11]:
max(recall_list)

0.9787234042553191

In [12]:
max(precision_list)

0.9822064056939501

In [13]:
conv_angles = np.asarray(conv_angles)
step_sizes = np.asarray(step_sizes)
rmse_list = np.asarray(rmse_list)
precision_list = np.asarray(precision_list)
recall_list = np.asarray(recall_list)

In [14]:
conv_angles_reshaped = conv_angles.reshape(8,8)
conv_angles_vals = conv_angles_reshaped[0]

In [15]:
conv_angles_vals

array([0.016, 0.02 , 0.024, 0.03 , 0.04 , 0.05 , 0.064, 0.084])

In [16]:
step_sizes_reshaped = step_sizes.reshape(8,8)


In [17]:
probe_overlaps = np.asarray([0, 5, 15, 35, 60, 70, 80, 90])

In [18]:
rmse_vals = rmse_list.reshape(8,8)
precision_vals = precision_list.reshape(8,8)
recall_vals = recall_list.reshape(8,8)

In [45]:
import numpy as np
import matplotlib
import matplotlib.pyplot as plt

fig, ax = plt.subplots()
im = ax.imshow(rmse_vals, cmap = 'RdBu')

# We want to show all ticks...
ax.set_xticks(np.arange(len(conv_angles_vals)))
ax.set_yticks(np.arange(len(probe_overlaps)))
# ... and label them with the respective list entries
ax.set_xticklabels(conv_angles_vals)
ax.set_yticklabels(probe_overlaps)

# Rotate the tick labels and set their alignment.
plt.setp(ax.get_xticklabels(), rotation=45, ha="right",
         rotation_mode="anchor")
plt.xlabel('convergence angle (rad)')
plt.ylabel('probe real space overlap (%)')

# Loop over data dimensions and create text annotations.
for i in range(len(conv_angles_vals)):
    for j in range(len(probe_overlaps)):
        text = ax.text(j, i, np.round(rmse_vals[i, j],2),
                       ha="center", va="center", color="w")

ax.set_title("RMSE values")
fig.tight_layout()
plt.show()

In [30]:
import numpy as np
import matplotlib
import matplotlib.pyplot as plt

fig, ax = plt.subplots(1,3, sharex=True, sharey=True)
ax[0].imshow(rmse_vals, cmap = 'RdBu')

plt.xlabel('convergence angle (rad)')
plt.ylabel('probe real space overlap (%)')

ax[1].imshow(precision_vals, cmap = 'RdBu')
ax[2].imshow(recall_vals, cmap = 'RdBu')

# We want to show all ticks...
ax[0].set_xticks(np.arange(len(conv_angles_vals)))
ax[0].set_yticks(np.arange(len(probe_overlaps)))
# ... and label them with the respective list entries
ax[0].set_xticklabels(conv_angles_vals)
ax[0].set_yticklabels(probe_overlaps)



# Rotate the tick labels and set their alignment.
plt.setp(ax[0].get_xticklabels(), rotation=45, ha="right",
         rotation_mode="anchor")
plt.setp(ax[1].get_xticklabels(), rotation=45, ha="right",
         rotation_mode="anchor")
plt.setp(ax[2].get_xticklabels(), rotation=45, ha="right",
         rotation_mode="anchor")


# Loop over data dimensions and create text annotations.
for i in range(len(conv_angles_vals)):
    for j in range(len(probe_overlaps)):
        text = ax[0].text(j, i, np.round(rmse_vals[i, j],2),
                       ha="center", va="center", color="w")
        text = ax[1].text(j, i, np.round(precision_vals[i, j],2),
                       ha="center", va="center", color="w")
        text = ax[2].text(j, i, np.round(recall_vals[i, j],2),
                       ha="center", va="center", color="w")

ax[0].set_title("RMSE values")
ax[1].set_title("Precision values")
ax[2].set_title("Recall values")
fig.tight_layout()
plt.show()

# Trial on three example datasets

In [95]:
data_file_good = '/dls/science/groups/e02/Mohsen/code/Git_Repos/Staff-notebooks/ptyREX_sim_matrix/fitting_results/graphene_512_64matrix/Graphene_defect_25.0mrad_181.36A_def_2.17A_step_size.h5'
data_dict_good = load_dict_from_hdf5(data_file_good)
data_file_fair = '/dls/science/groups/e02/Mohsen/code/Git_Repos/Staff-notebooks/ptyREX_sim_matrix/fitting_results/graphene_512_64matrix/Graphene_defect_42.0mrad_144.04A_def_2.17A_step_size.h5'
data_dict_fair = load_dict_from_hdf5(data_file_fair)
data_file_poor = '/dls/science/groups/e02/Mohsen/code/Git_Repos/Staff-notebooks/ptyREX_sim_matrix/fitting_results/graphene_512_64matrix/Graphene_defect_8.0mrad_281.75A_def_21.77A_step_size.h5'
data_dict_poor = load_dict_from_hdf5(data_file_poor)

In [ ]:
good = data_dict_good['atom_pos'][:,:2]
fair = data_dict_fair['atom_pos'][:,:2]
poor = data_dict_poor['atom_pos'][:,:2]
truth = np.load('/dls/science/groups/e02/Mohsen/code/Git_Repos/Staff-notebooks/ptyREX_sim_matrix/ground_truth_positions.npy')
truth_pos = truth[:,:2]

In [121]:
data_dict = data_dict_good

plot_kdtree_results(data_dict['NN_Results']['TP_list'],
                   data_dict['NN_Results']['FN_list'],
                   data_dict['NN_Results']['FP_list'],
                   unlabeled_experiment=data_dict['atom_pos'][:,:2],
                    truth=truth_pos)

IndexError: too many indices for array

In [ ]:
plot_kdtree_results()

In [112]:
t = data_dict['atom_pos'][:,:2]

In [118]:
t[0]

array([161.68821904, 164.40397512])

In [105]:
data_dict_good.keys()

dict_keys(['NN_Results', 'atom_pos', 'base_dir', 'experiment', 'json_path', 'process', 'sampling_factor', 'sim_path'])

In [98]:
good = data_dict_good['atom_pos'][:,:2]
fair = data_dict_fair['atom_pos'][:,:2]
poor = data_dict_poor['atom_pos'][:,:2]

In [99]:
type(good)

numpy.ndarray

In [8]:
import matplotlib.pyplot as plt
truth_arr = np.asarray(truth)
good_arr = np.asarray(good)
plt.figure()
plt.scatter(truth_arr[:,0], truth_arr[:,1], label = 'truth')
plt.scatter(good_arr[:,0], good_arr[:,1], label = 'exp')
# plt.scatter(false_pos[:,0], false_pos[:,1], label = 'false_pos')
# plt.scatter(false_neg[:,0], false_neg[:,1], label = 'false_neg')
plt.legend()

In [25]:
from sklearn.neighbors import KDTree
import numpy as np
search_rad = 2.
exp_list = list(poor)
truth_list = list(truth_pos)
false_neg = []
# we have atoms in truth_pos that have gone undetected in recon
false_pos = [] 
# detected an atom not present in truth_pos
paired_list = []
paired_dict = {}

distances = []
inds = []

for i in range(len(exp_list)):
    test = np.vstack((exp_list[i], truth_list))
    tree = KDTree(test, leaf_size=10)
    [ind, d] = tree.query_radius(test[:1], r=search_rad, count_only=False, return_distance = True)
    
    if len(ind[0]) == 1:
        false_pos.append(exp_list[i])
    elif len(ind[0]) == 2:
        inds.append([e-1 for e in ind[0] if e != 0])
#         paired_dict['atom ' + str(i)] = truth_list.pop(inds[-1][0])
        distances.append([e for e in d[0] if e != 0])
        atom_entry = list([truth_list.pop(inds[-1][0]), [exp_list[i], distances[-1]]])
        paired_list.append(atom_entry)

    del(test)
TP = len(paired_list)
print('TP: ', len(paired_list))
false_pos = np.asarray(false_pos)
FP = len(false_pos)
print('FP: ', len(false_pos))
false_neg = np.asarray(truth_list)
FN = len(false_neg)
print('FN: ', len(false_neg))

precision = TP / (TP + FP)
recall = TP / (TP + FN)

print('precision: ', precision)
print('recall: ', recall)
print('RMSE: ', get_RMSE(distances))

TP:  47
FP:  224
FN:  235
precision:  0.17343173431734318
recall:  0.16666666666666666
RMSE:  4.696551537163271


In [44]:
distances

[[1.813620242193791],
 [1.5240340549740834],
 [1.8158794473964452],
 [1.8918504239565088],
 [1.924412955214493],
 [1.0877746702820588],
 [1.888554457483371],
 [0.6883910758359352],
 [0.19851619319172334],
 [1.378038032807637],
 [0.8940102395976366],
 [1.9068711380041232],
 [1.697555206522311],
 [1.3480604839508514],
 [1.4982839130722612],
 [1.0365523807629742],
 [1.1094273551780591],
 [1.1656045897172729],
 [1.9318610629409174],
 [1.6723768141168398],
 [1.1605140764529431],
 [0.40437386604805103],
 [1.613268538698602],
 [1.421998215577083],
 [1.0803711483420237],
 [1.5452792070518802],
 [1.6160567210403405],
 [1.5572063237842462],
 [1.2697243345458555],
 [1.3968424372183674],
 [1.8441915545188545],
 [1.3479695637911615],
 [1.6066363888910158],
 [1.9878233192958],
 [0.6284453855924453],
 [1.5375351467389697],
 [1.4224620575007696],
 [1.7658292689060713],
 [1.9947522771351225],
 [1.1367729222186793],
 [1.4532954790605834],
 [1.5754557479591076],
 [0.3396763319442411],
 [1.928918404540200

In [49]:
distances = np.asarray(distances)
rmse = np.square((distances ** 2).mean())
print(rmse)

4.696551537163271


In [37]:
t = plot_kdtree_results(paired_list, false_neg, false_pos, truth_arr, np.asarray(poor))

'c' argument looks like a single numeric RGB or RGBA sequence, which should be avoided as value-mapping will have precedence in case its length matches with 'x' & 'y'.  Please use a 2-D array with a single row if you really want to specify the same RGB or RGBA value for all points.


In [182]:
get_distance((41.06,53.65),(41.79,54.38))

1.03237590053236

In [112]:
plt.figure()
plt.scatter(test[:,0], test[:,1])
plt.scatter(truth_pos[1][0], truth_pos[1][1])

In [47]:
import numpy as np
rng = np.random.RandomState(0)
X = rng.random_sample((20, 2))  # 10 points in 3 dimensions
tree = KDTree(X, leaf_size=2)              # doctest: +SKIP
dist, ind = tree.query(X[:1], k=2)                # doctest: +SKIP
print(ind)  # indices of 3 closest neighbors

print(dist)  # distances to 3 closest neighbors


[[ 0 11]]
[[0.         0.10907128]]


In [53]:
import numpy as np
rng = np.random.RandomState(0)
X = rng.random_sample((10, 2))  # 10 points in 3 dimensions
tree = KDTree(X, leaf_size=2)     # doctest: +SKIP
print(tree.query_radius(X[:1], r=0.2, count_only=True))

ind = tree.query_radius(X[:1], r=0.2)  # doctest: +SKIP
print(ind)  # indices of neighbors within distance 0.3


[3]
[array([2, 0, 1])]


In [50]:
X

array([[0.5488135 , 0.71518937],
       [0.60276338, 0.54488318],
       [0.4236548 , 0.64589411],
       [0.43758721, 0.891773  ],
       [0.96366276, 0.38344152],
       [0.79172504, 0.52889492],
       [0.56804456, 0.92559664],
       [0.07103606, 0.0871293 ],
       [0.0202184 , 0.83261985],
       [0.77815675, 0.87001215]])